# Loan Default Prediction

# Group 4

# Group Members -

# Anuksha jain - 015002
# Ayushi Gupta - 015005
# Aniket Nandy - 015015
# Biswadip Das - 015025
# Siddharth Gupta - 015045

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import time
warnings.filterwarnings("ignore")
# Splitting data into training and testing
from sklearn.model_selection import train_test_split


In [88]:
train=pd.read_csv(r"C:\Users\nandy\Downloads\train_v2.csv")

In [89]:
train.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,1,126,10,0.686842,1100,3,13699,7201.0,4949.0,126.75,...,5,2.14,-1.54,1.18,0.1833,0.7873,1,0,5,0
1,2,121,10,0.782776,1100,3,84645,240.0,1625.0,123.52,...,6,0.54,-0.24,0.13,0.1926,-0.6787,1,0,5,0
2,3,126,10,0.500080,1100,3,83607,1800.0,1527.0,127.76,...,13,2.89,-1.73,1.04,0.2521,0.7258,1,0,5,0
3,4,134,10,0.439874,1100,3,82642,7542.0,1730.0,132.94,...,4,1.29,-0.89,0.66,0.2498,0.7119,1,0,5,0
4,5,109,9,0.502749,2900,4,79124,89.0,491.0,122.72,...,26,6.11,-3.82,2.51,0.2282,-0.5399,0,0,5,0


In [90]:
train.shape

(105471, 771)

In [91]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105471 entries, 0 to 105470
Columns: 771 entries, id to loss
dtypes: float64(653), int64(99), object(19)
memory usage: 620.4+ MB


In [92]:
train.isnull().sum()

id         0
f1         0
f2         0
f3         0
f4         0
        ... 
f775    1525
f776       0
f777       0
f778       0
loss       0
Length: 771, dtype: int64

In [93]:
train.shape

(105471, 771)

In [94]:
# # Missing Values

# Function to calculate missing values by column
def total_missing_values(train):
        # Total missing values
        count_nulls = train.isnull().sum()
        
        # Percentage of missing values
        null_percentage = 100 * train.isnull().sum() / len(train)
        
        # Make a table with the results
        null_data = pd.concat([count_nulls, null_percentage], axis=1)
        
        # Rename the columns
        null_data_ren_columns = null_data.rename(columns = {0 : 'null values', 1 : 'null_percent'})
        
        null_data_ren_columns = null_data_ren_columns[
        null_data_ren_columns.iloc[:,1] != 0].sort_values('null_percent', ascending=False).round(1)
        
        # Print some information
        print ('Train is having ' + str(train.shape[1]) + " columns"      
            " and " + str(null_data_ren_columns.shape[0]) +
              " columns with null values.")
       
        return null_data_ren_columns

In [95]:
total_missing_values(train).head(10)

Train is having 771 columns and 525 columns with null values.


,null values,null_percent
f662,18833,17.9
f663,18833,17.9
f159,18736,17.8
f160,18736,17.8
f170,18417,17.5
f169,18417,17.5
f618,18407,17.5
f619,18407,17.5
f331,18067,17.1
f330,18067,17.1


In [96]:
train.fillna(train.mean(), inplace=True)

In [97]:
total_missing_values(train).head(10)

Train is having 771 columns and 12 columns with null values.


,null values,null_percent
f206,1291,1.2
f207,1291,1.2
f390,698,0.7
f391,698,0.7
f626,698,0.7
f627,698,0.7
f695,698,0.7
f698,698,0.7
f138,182,0.2
f137,177,0.2


In [98]:
train.dropna(inplace=True)
total_missing_values(train)

Train is having 771 columns and 0 columns with null values.


,null values,null_percent


In [99]:
train.shape

(103302, 771)

In [100]:
#Exploratory Data Analysis

In [105]:
# # Correlations between Features and Target
corr=train.corr()
corr['loss'].sort_values(ascending=False).head(10)

loss    1.000000
f471    0.039538
f536    0.026087
f674    0.018999
f67     0.015012
f670    0.014811
f597    0.014165
f599    0.014165
f68     0.013375
f13     0.011933
Name: loss, dtype: float64

In [106]:
corr['loss'].sort_values(ascending = False,na_position ='first').tail(10)

f738   -0.008635
f734   -0.009284
f322   -0.009299
f69    -0.010269
f323   -0.010676
f314   -0.010689
f70    -0.010740
f315   -0.011106
f776   -0.015111
f612   -0.016943
Name: loss, dtype: float64

In [102]:
train.shape

(103302, 771)

In [118]:
Fraud = train.loc[train['loss']==1]

Valid = train.loc[train['loss']==0]

fraction = len(Fraud)/(len(Fraud)+len(Valid))

print("Valid: ",len(Valid))

print("Fraud: ",len(Fraud))

print("fraction of fraud: ",fraction)

Valid:  93660
Fraud:  1134
fraction of fraud:  0.011962782454585734


In [121]:
# Remove the collinear features above a specified correlation coefficient
train = remove_collinear_features(train, 0.6);

In [122]:
train.shape

(103302, 186)

In [123]:
# # # Split Into Training and Testing Sets

# Separate out the features and targets
features = train.drop(columns='loss')
targets = pd.DataFrame(train['loss'])

# Split into 80% training and 20% testing set
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size = 0.2, random_state = 42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(82641, 185)
(20661, 185)
(82641, 1)
(20661, 1)


In [124]:
# # Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [125]:
# Convert y to one-dimensional array (vector)
y_train = np.array(y_train).reshape((-1, ))
y_test = np.array(y_test).reshape((-1, ))

In [126]:
X_train

array([[-1.06877109, -0.16688215,  1.03564494, ...,  0.        ,
         0.65159233, -0.66443318],
       [-0.48425848,  2.15138579, -0.14682708, ...,  0.        ,
         0.74503266, -0.66443318],
       [-0.43576466,  1.60591098, -0.73806309, ...,  0.        ,
         1.24726226,  1.50504223],
       ...,
       [ 0.84197055, -0.57598826,  1.03564494, ...,  0.        ,
        -0.75457294,  1.50504223],
       [-1.69955094, -0.64417261,  0.44440893, ...,  0.        ,
        -0.4115703 , -0.66443318],
       [-1.20154789,  1.06043617, -0.14682708, ...,  0.        ,
         0.87515049, -0.66443318]])

In [127]:
X_test

array([[-0.47027681, -1.18964742,  1.03564494, ...,  0.        ,
         4.19883151,  1.50504223],
       [ 0.34727464,  1.67409533, -0.73806309, ...,  0.        ,
         0.76570014, -0.66443318],
       [ 1.02805453, -0.3714352 , -2.51177113, ...,  0.        ,
         1.03078733,  1.50504223],
       ...,
       [ 0.29720911,  0.92406747, -0.73806309, ...,  0.        ,
        -0.70741614, -0.66443318],
       [ 0.85975052, -0.91691001,  1.03564494, ...,  0.        ,
        -0.90497014,  1.50504223],
       [ 0.00752321, -1.18964742, -1.3292991 , ...,  0.        ,
         0.69224809, -0.66443318]])

In [128]:
# Model Building
 # RandomForest with Bayes optimization
# xgboost with Bayes optimization
# Lightgbm with Bayes optimization

In [129]:
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [130]:
#Bayesian optimization
def bayesian_optimization(dataset, function, parameters):
   X_train, y_train, X_test, y_test = dataset
   n_iterations = 5
   gp_params = {"alpha": 1e-4}

   BO = BayesianOptimization(function, parameters)
   BO.maximize(n_iter=n_iterations, **gp_params)

   return BO.max

In [131]:
def rfc_optimization(cv_splits):
    def function(n_estimators, max_depth, min_samples_split):
        return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),                                                               
                   max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)), 
                   n_jobs=-1, 
                   random_state=42,   
                   class_weight="balanced"),  
               X=X_train, 
               y=y_train, 
               cv=cv_splits,
               scoring="roc_auc",
               n_jobs=-1).mean()

    parameters = {"n_estimators": (10, 1000),
                  "max_depth": (1, 150),
                  "min_samples_split": (2, 10)}
    
    return function, parameters

In [132]:
def xgb_optimization(cv_splits, eval_set):
    def function(eta, gamma, max_depth):
            return cross_val_score(
                   xgb.XGBClassifier(
                       objective="binary:logistic",
                       learning_rate=max(eta, 0),
                       gamma=max(gamma, 0),
                       max_depth=int(max_depth),                                               
                       seed=42,
                       nthread=-1,
                       scale_pos_weight = len(y_train[y_train == 0])/
                                          len(y_train[y_train == 1])),  
                   X=X_train, 
                   y=y_train, 
                   cv=cv_splits,
                   scoring="roc_auc",
                   fit_params={
                        "early_stopping_rounds": 10, 
                        "eval_metric": "auc", 
                        "eval_set": eval_set},
                   n_jobs=-1).mean()

    parameters = {"eta": (0.001, 0.4),
                  "gamma": (0, 20),
                  "max_depth": (1, 2000)}
    
    return function, parameters

In [133]:
#Train model
def train(X_train, y_train, X_test, y_test, function, parameters):
    dataset = (X_train, y_train, X_test, y_test)
    cv_splits = 4
    
    best_solution = bayesian_optimization(dataset, function, parameters)      
    params = best_solution["params"]

    model = RandomForestClassifier(
             n_estimators=int(max(params["n_estimators"], 0)),
             max_depth=int(max(params["max_depth"], 1)),
             min_samples_split=int(max(params["min_samples_split"], 2)), 
             n_jobs=-1, 
             random_state=42,   
             class_weight="balanced")

    model.fit(X_train, y_train)
    
    return model

In [134]:
import lightgbm as lgb
from bayes_opt import BayesianOptimization

train = sns.load_dataset('diamonds')

train["color"] = train["color"].astype('category')
train["color_cat"] = train["color"].cat.codes
train = train.drop(["color"],axis = 1)

train["cut"] = train["cut"].astype('category')
train["cut_cat"] = train["cut"].cat.codes
train = train.drop(["cut"],axis = 1)

train["clarity"] = train["clarity"].astype('category')
train["clarity_cat"] = train["clarity"].cat.codes
train = train.drop(["clarity"],axis = 1)

y = train['price']
X = train.drop(['price'], axis=1)

seed = 7
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=seed)

train_lgb = lgb.Dataset(X_train, y_train)
eval_lgb = lgb.Dataset(X_test, y_test, reference = train_lgb)

params = { 'objective': 'regression',
  'metric': 'RMSE',
  'learning_rate': 0.02}
lgb_reg = lgb.train(params, train_lgb, num_boost_round = 10000, early_stopping_rounds=50, verbose_eval = 100, valid_sets=eval_lgb)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001723 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1267
[LightGBM] [Info] Number of data points in the train set: 37758, number of used features: 9
[LightGBM] [Info] Start training from score 3935.724588
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 857.111
[200]	valid_0's rmse: 576.605
[300]	valid_0's rmse: 546.569
[400]	valid_0's rmse: 540.681
[500]	valid_0's rmse: 538.675
[600]	valid_0's rmse: 537.972
Early stopping, best iteration is:
[613]	valid_0's rmse: 537.86


In [135]:
def modelFitter(colsampleByTree, subsample,maxDepth, num_leaves):   
    model = lgb.LGBMRegressor(learning_rate=0.02, n_estimators=10000, max_depth=maxDepth.astype("int32"), subsample=subsample, colsample_bytree=colsampleByTree,num_leaves=num_leaves.astype("int32"))

    evalSet  = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_metric="rmse", eval_set=evalSet, early_stopping_rounds=50, verbose=False)

    bestScore = model.best_score_[list(model.best_score_.keys())[0]]['rmse']

    return -bestScore

# Bounded region of parameter space
pbounds = {'colsampleByTree': (0.8,1.0), 'subsample': (0.8,1.0), 'maxDepth': (2,5), 'num_leaves': (24, 45)}

optimizer = BayesianOptimization(
    f=modelFitter,
    pbounds=pbounds,
    random_state=1)

optimizer.maximize(init_points=5,n_iter=5)  #n_iter=bayesian, init_points=random

|   iter    |  target   | colsam... | maxDepth  | num_le... | subsample |
-------------------------------------------------------------------------
|  1        | -541.4    |  0.8834   |  4.161    |  24.0     |  0.8605   |
|  2        | -628.4    |  0.8294   |  2.277    |  27.91    |  0.8691   |
|  3        | -565.2    |  0.8794   |  3.616    |  32.8     |  0.937    |
|  4        | -541.4    |  0.8409   |  4.634    |  24.58    |  0.9341   |
|  5        | -565.2    |  0.8835   |  3.676    |  26.95    |  0.8396   |
|  6        | -628.4    |  0.8018   |  2.036    |  24.0     |  0.8074   |
|  7        | -544.8    |  0.9847   |  4.934    |  26.07    |  0.8586   |
|  8        | -541.4    |  0.9301   |  4.725    |  35.8     |  0.9746   |
|  9        | -624.3    |  0.8398   |  2.771    |  36.43    |  0.9047   |
|  10       | -544.8    |  0.9825   |  4.996    |  34.43    |  0.8723   |
